In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# import cv2
# import numpy as np

# # List to store the points
# points = []

# # Callback function to capture the points
# def select_point(event, x, y, flags, param):
#     if event == cv2.EVENT_LBUTTONDOWN:
#         points.append((x, y))
#         # Draw the point on the image
#         cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
#         cv2.imshow("Image", image)
#         if len(points) == 2:
#             cv2.line(image, points[0], points[1], (255, 0, 0), 2)
#             cv2.imshow("Image", image)
#             return

# # Load the image
# # image_path = "../data/children/stimuli_R/1005.png"
# image_path = "../stimuli/transparent_mask/stimuli_G.png"
# image = cv2.imread(image_path)

# # Create a window and set a mouse callback function
# cv2.imshow("Image", image)
# cv2.setMouseCallback("Image", select_point)

# # Wait until the user has selected 2 points
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# # Calculate the distance if two points are selected
# if len(points) == 2:
#     point1, point2 = points
#     distance = np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
#     print(f"The distance between the points is: {distance:.2f} pixels")

#     # Optionally, save the image with the points and line
#     output_path = "/mnt/data/marked_image.png"
#     cv2.imwrite(output_path, image)
# else:
#     print("Please select exactly two points.")

In [62]:
extra = 38
def crop_image(image, l):
    top, bottom, left, right = np.array(l) - extra
    height, width = image.shape[:2]
    return image[top:height - bottom, left:width - right]

def find_crop_sizes(stim, inv):
    if stim == "G" and inv == False:
        return [38, 184, 110, 385]
    elif stim == "G" and inv == True:
        return [52, 200, 138, 378]
    elif stim == "I" and inv == False:
        return [48, 189, 115, 395]
    elif stim == "I" and inv == True:
        return [39, 192, 133, 373]
    elif stim == "R" and inv == True:
        return [52, 202, 117, 393]
    elif stim == "R" and inv == False:
        return [54, 198, 133, 377]

df = pd.read_csv("../csvs/product.csv")

for index, row in df.iterrows():
    if row["creator"] != "adult":
        continue
    
    # Load the image
    image_path = row["filepath"][:8] + "_" + row["filepath"][8:]
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    _, binary_image = cv2.threshold(image, 230, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask to remove small dots
    mask = np.ones(binary_image.shape[:2], dtype="uint8") * 255

    # Loop over the contours
    for cnt in contours:
        # If the contour is small, fill it in on the mask
        if cv2.contourArea(cnt) < 20:  # Adjust this threshold value as needed
            cv2.drawContours(mask, [cnt], -1, 0, -1)

    # Use the mask to remove small dots from the original image
    cleaned_image = cv2.bitwise_and(binary_image, binary_image, mask=mask)

    
    small_kernel = np.ones((1,1),np.uint8)
    # Erode the image slightly to remove small noise
    eroded_image = cv2.erode(cleaned_image, small_kernel, iterations=3)

    # # Dilation to thicken the lines
    if row["creator"] == "adult":
        kernel = np.ones((2,2), np.uint8)
        dilated_image = cv2.dilate(cleaned_image, kernel, iterations=1)
    
    if row["creator"] == "child":
        kernel = np.ones((4,4), np.uint8)
        dilated_image = cv2.dilate(cleaned_image, kernel, iterations=2)
    
    # # Inverting the image back to get black lines on a white background
    inverted_image = cv2.bitwise_not(dilated_image)

    if row["creator"] == "adult":
        final_image = inverted_image
    
    if row["creator"] == "child":
        original_height, original_width = image.shape[:2]
        new_width = int(original_width / 2.77)
        new_height = int(original_height * (new_width / original_width))
        resized_image = cv2.resize(inverted_image, (new_width, new_height))
        final_image = crop_image(resized_image, find_crop_sizes(row["stim"], row["inverted"]))

    # # Display the final image
    # plt.imshow(final_image, cmap='gray')
    # plt.axis('off')
    # plt.show()

    # Save the final image
    output_path = row["filepath"]
    cv2.imwrite(output_path, final_image)
